In [20]:
import pandas
import numpy
import bs4

TD = pandas.read_csv('cleaned_trials_data.csv')

Several issues found:

1. fix the dataframe index to given Rank
2. Since results is a boolean variable, convert from string to boolean
3. Same as 2 for Volunteers variable
4. ? Possible change EligibleSexes to binary distribution with all sexes at 0
5. A) Filling NaN values of Exclusion Criteria with empty strings
5. B) Remove HTML tags from Inclusion and Exclusion Criteria
6. Splitting Criteria text apart to create a malleable bag of words

In [21]:
#Use the rank as the index instead
TD.set_index(TD.Rank)

#Use previous resultBool from analyzing SR
resultBool = {'No Results Available': 0, 'Has Results': 1}
TD['Contains Results'] = TD['Study Results'].map(resultBool)

#change volunteer status from yes/no to boolean
TD['Volunteers'] = TD['AcceptsHealthyVolunteers'].map({'Yes' : 1, 'No' : 0})

#possibly change EligibleSexes to binary with All == 0
#TD.Sex = TD.EligibleSexes.map({'All' : 0, 'Female' : 1, 'Male' : -1})

#remove HTML tags from In/ExclusionCriteria
from bs4 import BeautifulSoup
TD['InCrit'] = TD['InclusionCriteria'].map(lambda x: BeautifulSoup(x).get_text().replace(',', '').lower().split())
TD['ExCrit'] = TD['ExclusionCriteria'].fillna('').map(lambda y: BeautifulSoup(y).get_text().replace(',', '').lower().split())
TD

,Rank,Title,Status,Study Results,Conditions,Interventions,Locations,URL,EligibleAges,EligibleSexes,...,Inclusion People Topic Criteria,Exclusion People Topic Criteria,bagInclusion,bagExclusion,numWordsInclusion,numWordsExclusion,Contains Results,Volunteers,InCrit,ExCrit
0,1,A Safety and Efficacy Study to Evaluate Wound ...,Recruiting,No Results Available,Wound Heal,Drug: AIV001,"Cosmetic Laser Dermatology, San Diego, Califor...",https://ClinicalTrials.gov/show/NCT03639883,21 Years to 55 Years,All,...,['Smoking'],[],"['eligible', 'for', 'an', 'abdominoplasty', 'n...","['existing', 'scars', 'in', 'study', 'area', '...",9,43,0,1.0,"[eligible, for, an, abdominoplastynonsmokerfit...","[existing, scars, in, study, area, active, inf..."
1,2,Wound Eversion Versus Planar Closure for Face ...,Completed,No Results Available,Operative Wound,Other: Everted closure|Other: Planar closure,"University of California, Davis, Department of...",https://ClinicalTrials.gov/show/NCT03302988,18 Years and older,All,...,['Consent'],"['Language', 'Pregnancy', 'Consent', 'Pregnanc...","['>18', 'years', 'of', 'age', 'able', 'to', 'g...","['mentally', 'handicapped', 'unable', 'to', 'u...",32,35,0,0.0,"[over, 18, years, of, ageable, to, give, infor...","[mentally, handicappedunable, to, understand, ..."
2,3,Single-Use Negative Pressure Wound Therapy for...,"Active, not recruiting",No Results Available,Surgical Wound,Device: PICO Single Use Negative Pressure Woun...,"University of Alabama at Birmingham, Birmingha...",https://ClinicalTrials.gov/show/NCT03340025,18 Years and older,All,...,['Consent'],['Consent'],"['>18', 'years', 'of', 'age', 'participants', ...","['<18', 'years', 'of', 'age', 'participants', ...",37,28,0,0.0,"[>, 18, years, of, ageparticipants, are, able,...","[<, 18, years, of, ageparticipants, are, unabl..."
3,4,Topical Collagen Powder for Healing of Acute F...,Completed,No Results Available,Wound of Skin|Wound Heal,Device: Nuvagen Collagen Powder,"George Washington Medical Faculty Associates, ...",https://ClinicalTrials.gov/show/NCT03481907,18 Years to 75 Years,All,...,"['Consent', 'Outpatient']","['Pregnancy', 'Alcohol']","['outpatient', '', 'male', 'or', 'female', 'su...","['patients', 'that', 'have', 'any', 'medical',...",55,291,0,1.0,"[outpatient, male, or, female, subjects, of, a...","[patients, that, have, any, medical, or, skin,..."
4,5,PROphylactic Wound VACuum Therapy to Decrease ...,Completed,Has Results,Wound Complication,Device: Closed incision wound vacuum (Prevena),"University of Pennsylvania, Philadelphia, Penn...",https://ClinicalTrials.gov/show/NCT02128997,18 Years to 60 Years,Female,...,[],[],[''],[''],1,1,1,1.0,[]],[]
5,6,A Retrospective Analysis of Laboratory Testing...,Completed,No Results Available,Wound Chronic Draining,NaN,"Southwest Regional Wound Care Center, Lubbock,...",https://ClinicalTrials.gov/show/NCT02142426,"Child, Adult, Older Adult</td",All,...,[],[],[''],[''],1,1,0,0.0,[]],[]
6,7,Cytal® Wound Matrix and MicroMatrix® Wound Study,Recruiting,No Results Available,Wounds,Device: ACell Arm,"NYU Winthrop Hospital, Long Island City, New Y...",https://ClinicalTrials.gov/show/NCT03632954,21 Years and older,All,...,[],[],[''],[''],1,1,0,0.0,[]],[]
7,8,Doubleblinded RCT of the Omnilux on Lower Extr...,Terminated,Has Results,Wound,Device: Omnilux|Device: Omnilux Sham,University of Miami Hospital and Clinics/Sylve...,https://ClinicalTrials.gov/show/NCT02383056,18 Years to 90 Years,All,...,[],['Pregnancy'],"['surgical', 'defect', 'on', 'the', 'lower', '...","['history', 'of', 'porphyria', 'photosensitive...",38,59,1,0.0,"[surgical, defect, on, the, lower, extremity, ...","[history, of, porphyria, photosensitive, erupt..."
8,9,Collection of Samples for the Evaluation of Wo...,Completed,No Results Available,Wounds,Other: no treatment,"Southwest Regional Wound Care Center, Lubbock,...",https://ClinicalTrials.gov/show/NCT00764296,18 Years and older,All,...,['Reading and writing skills'],"['Pregnancy', 'Therapy']","['the', 

In [32]:
import itertools

#g = TD.groupby('Locations')
#incs = g['CleanedInclusionCriteria'].aggregate(lambda listofbags: list(itertools.chain.from_iterable(listofbags)))
g = TD.groupby('Conditions')
cond = g['Conditions'].aggregate(lambda listofbags: list(itertools.chain.from_iterable(listofbags)))
TF = incs.iloc[:1000].apply(lambda bag: pandas.Series(bag).value_counts())
#IDF = numpy.log(len(TF)/TF.count())
#TFIDF = TF * IDF
#TFIDF
TF

,,e,a,t,i,s,n,o,c,r,...,±,β,ü,ï,•,½,µ,₂,º,²
Locations,,,,,,,,,,,,,,,,,,,,,
"110, Montgomery, Alabama, United States|103, Anaheim, California, United States|117, Long Beach, California, United States|106, Long Beach, California, United States|118, Modesto, California, United States|104, San Diego, California, United States|113, San Diego, California, United States|115, San Diego, California, United States|116, San Diego, California, United States|108, Stockton, California, United States|105, Sylmar, California, United States|112, Washington, District of Columbia, United States|107, Orlando, Florida, United States|120, Saint Cloud, Florida, United States|114, Augusta, Georgia, United States|122, Columbus, Georgia, United States|125, Savannah, Georgia, United States|119, Eunice, Louisiana, United States|101, Springfield, Massachusetts, United States|109, Detroit, Michigan, United States|121, Butte, Montana, United States|123, Omaha, Nebraska, United States|111, Toledo, Ohio, United States|126, Franklin, Tennessee, United States|127, Smyrna, Tennessee, United States|802, Sofia, Bulgaria|800, Sofia, Bulgaria|801, Sofia, Bulgaria|200, Zagreb, Croatia|201, Zagreb, Croatia|253, Tallinn, Estonia|252, Tallinn, Estonia|251, Tartu, Estonia|302, Kutaisi, Georgia|303, Tbilisi, Georgia|300, Tbilisi, Georgia|301, Tbilisi, Georgia|352, Debrecen, Hungary|353, Kaposvar, Hungary|354, Pecs, Hungary|351, Szeged, Hungary|402, Daugavpils, Latvia|401, Liepaja, Latvia|403, Rezekne, Latvia|400, Riga, Latvia|404, Riga, Latvia|501, Cluj-Napoca, Cluj County, Romania|502, Bucharest, Romania|500, Bucharest, Romania|503, Bucharest, Romania|555, Vsevolozhsk, Leningrad Region, Russian Federation|557, Irkutsk, Russian Federation|552, Moscow, Russian Federation|554, Moscow, Russian Federation|553, Novosibirsk, Russian Federation|551, St. Petersburg, Russian Federation|556, Tomsk, Russian Federation|600, Belgrade, Serbia|601, Belgrade, Serbia|603, Nis, Serbia|602, Novi Sad, Serbia|756, Breyten, South Africa|760, Cape Town, South Africa|752, Dundee, South Africa|755, Johannesburg, South Africa|751, Middleburg, South Africa|758, Port Elizabeth, South Africa|757, Pretoria, South Africa|753, Pretoria, South Africa|759, Pretoria, South Africa|754, Worcester, South Africa|700, Cherkasy, Ukraine|704, Dnipropetrovsk, Ukraine|706, Ivano-Frankivsk, Ukraine|701, Ivano-Frankivsk, Ukraine|705, Kharkiv, Ukraine|703, Lviv, Ukraine|702, Zaporizhzhya, Ukraine",73.0,34.0,33.0,33.0,33.0,31.0,28.0,25.0,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1199 Norc, New York, New York, United States|Warbasse Cares NORC, New York, New York, United States|Rochdale Village NORC, New York, New York, United States",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"3C Institute, Durham, North Carolina, United States",57.0,37.0,14.0,29.0,19.0,13.0,15.0,27.0,9.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"72d Medical Group, Oklahoma City, Oklahoma, United States",12.0,4.0,5.0,5.0,12.0,6.0,6.0,6.0,4.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"921 Assembly Street, 3rd Floor, Public Health Research Center, University of South Carolina, Columbia, South Carolina, United States",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"96th Medical Group, Eglin Air Force Base, Florida, United States",55.0,38.0,21.0,22.0,32.0,17.0,15.0,20.0,10.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"96th Medical Group, Eglin Air Force Base, Florida, United States|Mike O'Callaghan Military Medical Center, Nellis Air Force Base, Nevada, United States",50.0,32.0,26.0,19.0,18.0,17.0,19.0,15.0,8.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"AAC Institute Clinic (ICAN Talk Clinic), Carnegie, Pennsylvania, United States|University of Pittsburgh, AAC Performance and Testing Laboratory, Pittsburgh, Pennsylvania, United States",50.0,35.0,28.0,20.0,25.0,17.0,25.0,17.0,11.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"AMITA Healt

In [29]:
TF.columns

Index([' ', 'e', 'a', 't', 'i', 's', 'n', 'o', 'c', 'r', 'd', 'l', ''', 'u',
       'm', 'f', 'p', 'g', 'b', ',', 'y', 'w', '8', 'k', '-', 'v', ']', '[',
       'h', '5', '2', 'j', '1', '>', 'x', '0', '3', 'z', '6', 'q', '4', ';',
       '"', '^', '/', '‐', '&', '%', '<', '≥', '9', '+', '7', '°', '≤', '\',
       '⁄', '×', '~', '=', '*', '#', '†', '±', 'β', 'ü', 'ï', '•', '½', 'µ',
       '₂', 'º', '²'],
      dtype='object')

In [33]:
TD

,Rank,Title,Status,Study Results,Conditions,Interventions,Locations,URL,EligibleAges,EligibleSexes,...,Inclusion People Topic Criteria,Exclusion People Topic Criteria,bagInclusion,bagExclusion,numWordsInclusion,numWordsExclusion,Contains Results,Volunteers,InCrit,ExCrit
0,1,A Safety and Efficacy Study to Evaluate Wound ...,Recruiting,No Results Available,Wound Heal,Drug: AIV001,"Cosmetic Laser Dermatology, San Diego, Califor...",https://ClinicalTrials.gov/show/NCT03639883,21 Years to 55 Years,All,...,['Smoking'],[],"['eligible', 'for', 'an', 'abdominoplasty', 'n...","['existing', 'scars', 'in', 'study', 'area', '...",9,43,0,1.0,"[eligible, for, an, abdominoplastynonsmokerfit...","[existing, scars, in, study, area, active, inf..."
1,2,Wound Eversion Versus Planar Closure for Face ...,Completed,No Results Available,Operative Wound,Other: Everted closure|Other: Planar closure,"University of California, Davis, Department of...",https://ClinicalTrials.gov/show/NCT03302988,18 Years and older,All,...,['Consent'],"['Language', 'Pregnancy', 'Consent', 'Pregnanc...","['>18', 'years', 'of', 'age', 'able', 'to', 'g...","['mentally', 'handicapped', 'unable', 'to', 'u...",32,35,0,0.0,"[over, 18, years, of, ageable, to, give, infor...","[mentally, handicappedunable, to, understand, ..."
2,3,Single-Use Negative Pressure Wound Therapy for...,"Active, not recruiting",No Results Available,Surgical Wound,Device: PICO Single Use Negative Pressure Woun...,"University of Alabama at Birmingham, Birmingha...",https://ClinicalTrials.gov/show/NCT03340025,18 Years and older,All,...,['Consent'],['Consent'],"['>18', 'years', 'of', 'age', 'participants', ...","['<18', 'years', 'of', 'age', 'participants', ...",37,28,0,0.0,"[>, 18, years, of, ageparticipants, are, able,...","[<, 18, years, of, ageparticipants, are, unabl..."
3,4,Topical Collagen Powder for Healing of Acute F...,Completed,No Results Available,Wound of Skin|Wound Heal,Device: Nuvagen Collagen Powder,"George Washington Medical Faculty Associates, ...",https://ClinicalTrials.gov/show/NCT03481907,18 Years to 75 Years,All,...,"['Consent', 'Outpatient']","['Pregnancy', 'Alcohol']","['outpatient', '', 'male', 'or', 'female', 'su...","['patients', 'that', 'have', 'any', 'medical',...",55,291,0,1.0,"[outpatient, male, or, female, subjects, of, a...","[patients, that, have, any, medical, or, skin,..."
4,5,PROphylactic Wound VACuum Therapy to Decrease ...,Completed,Has Results,Wound Complication,Device: Closed incision wound vacuum (Prevena),"University of Pennsylvania, Philadelphia, Penn...",https://ClinicalTrials.gov/show/NCT02128997,18 Years to 60 Years,Female,...,[],[],[''],[''],1,1,1,1.0,[]],[]
5,6,A Retrospective Analysis of Laboratory Testing...,Completed,No Results Available,Wound Chronic Draining,NaN,"Southwest Regional Wound Care Center, Lubbock,...",https://ClinicalTrials.gov/show/NCT02142426,"Child, Adult, Older Adult</td",All,...,[],[],[''],[''],1,1,0,0.0,[]],[]
6,7,Cytal® Wound Matrix and MicroMatrix® Wound Study,Recruiting,No Results Available,Wounds,Device: ACell Arm,"NYU Winthrop Hospital, Long Island City, New Y...",https://ClinicalTrials.gov/show/NCT03632954,21 Years and older,All,...,[],[],[''],[''],1,1,0,0.0,[]],[]
7,8,Doubleblinded RCT of the Omnilux on Lower Extr...,Terminated,Has Results,Wound,Device: Omnilux|Device: Omnilux Sham,University of Miami Hospital and Clinics/Sylve...,https://ClinicalTrials.gov/show/NCT02383056,18 Years to 90 Years,All,...,[],['Pregnancy'],"['surgical', 'defect', 'on', 'the', 'lower', '...","['history', 'of', 'porphyria', 'photosensitive...",38,59,1,0.0,"[surgical, defect, on, the, lower, extremity, ...","[history, of, porphyria, photosensitive, erupt..."
8,9,Collection of Samples for the Evaluation of Wo...,Completed,No Results Available,Wounds,Other: no treatment,"Southwest Regional Wound Care Center, Lubbock,...",https://ClinicalTrials.gov/show/NCT00764296,18 Years and older,All,...,['Reading and writing skills'],"['Pregnancy', 'Therapy']","['the', 

In [35]:
g = TD.groupby('Conditions')


In [37]:
TD.apply(
    lambda row: (TD != row).sum(axis = 1),
    axis = 1)

,0,1,2,3,4,5,6,7,8,9,...,5928,5929,5930,5931,5932,5933,5934,5935,5936,5937
0,0,26,26,24,25,24,23,28,26,23,...,26,26,23,22,23,25,23,25,24,24
1,26,0,21,25,28,23,24,25,22,21,...,25,23,25,27,24,25,23,25,23,23
2,26,21,0,26,29,24,24,25,23,22,...,26,24,26,27,24,26,23,25,21,22
3,24,25,26,0,26,25,26,28,24,25,...,27,24,24,25,26,24,26,27,26,26
4,25,28,29,26,1,13,14,25,28,26,...,22,28,22,12,14,23,14,22,25,14
5,24,23,24,25,13,2,9,24,22,21,...,21,23,22,12,9,23,9,21,20,9
6,23,24,24,26,14,9,1,24,23,22,...,22,24,23,11,8,25,8,22,20,9
7,28,25,25,28,25,24,24,0,25,26,...,22,26,27,26,24,25,24,22,24,24
8,26,22,23,24,28,22,23,25,0,22,...,25,23,25,27,24,26,23,25,23,23
9,23,21,22,25,26,21,22,26,22,0,...,23,23,23,25,22,26,21,23,20,21
